## Load Data

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
from itertools import product
from sklearn.metrics import precision_recall_fscore_support

In [9]:
# Define data transformations
train_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])

# Load the dataset
train_dataset = datasets.ImageFolder(root='melanoma_cancer_dataset/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='melanoma_cancer_dataset/test', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## CNN Model

In [10]:
class CNN(nn.Module):
    def __init__(self, num_classes=2, layers=[16, 32, 64]):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 3
        for out_channels in layers:
            self.layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(2, 2))
            in_channels = out_channels
        
        # Compute the size of the flattened features after all conv and pooling layers
        self.flattened_size = layers[-1] * (100 // 2**len(layers))**2
        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
num_epochs = [3, 5, 10]
layer_configs = [
    [16, 32],  # 2 layers
    [16, 32, 64],  # 3 layers
    [16, 32, 64, 128, 256]  # 5 layers
]

best_accuracy = 0
best_params = {}

for epochs, layers in product(num_epochs, layer_configs):
    # Initialize the model with the current layer configuration
    model = CNN(layers=layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    params = {
        'epochs': epochs,
        'layers': layers
    }

    # Train the model with the current number of epochs
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 20 == 19:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
                running_loss = 0.0
    
    #Testing of validation data
    true_labels = []
    predictions = []

    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predictions.extend(predicted.cpu().numpy())

    true_labels = np.array(true_labels)
    predictions = np.array(predictions)

    # Accuracy
    correct = (predictions == true_labels).sum().item()
    total = true_labels.size
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}%')

    # Precision, recall, and f1 score
    precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1_score}')
    print(f'Parameters: ', params)

    
    # Store the model if it has the best accuracy
    accuracy = 100 * correct / total  # Make sure 'correct' and 'total' are defined in your validation loop
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {
            'epochs': epochs,
            'layers': layers
        }

print(f"Best Accuracy: {best_accuracy}%")
print("Best Parameters:", best_params)

[1,    20] loss: 0.737
[1,    40] loss: 0.484
[1,    60] loss: 0.443
[1,    80] loss: 0.366
[1,   100] loss: 0.368
[1,   120] loss: 0.381
[1,   140] loss: 0.346
[1,   160] loss: 0.328
[1,   180] loss: 0.345
[1,   200] loss: 0.374
[1,   220] loss: 0.287
[1,   240] loss: 0.340
[1,   260] loss: 0.348
[1,   280] loss: 0.358
[1,   300] loss: 0.299
[2,    20] loss: 0.290
[2,    40] loss: 0.323
[2,    60] loss: 0.252
[2,    80] loss: 0.349
[2,   100] loss: 0.272
[2,   120] loss: 0.285
[2,   140] loss: 0.226
[2,   160] loss: 0.314
[2,   180] loss: 0.338
[2,   200] loss: 0.304
[2,   220] loss: 0.252
[2,   240] loss: 0.265
[2,   260] loss: 0.295
[2,   280] loss: 0.257
[2,   300] loss: 0.278
[3,    20] loss: 0.275
[3,    40] loss: 0.289
[3,    60] loss: 0.301
[3,    80] loss: 0.286
[3,   100] loss: 0.256
[3,   120] loss: 0.247
[3,   140] loss: 0.255
[3,   160] loss: 0.273
[3,   180] loss: 0.280
[3,   200] loss: 0.335
[3,   220] loss: 0.261
[3,   240] loss: 0.265
[3,   260] loss: 0.237
[3,   280] 

[3,   240] loss: 0.243
[3,   260] loss: 0.288
[3,   280] loss: 0.276
[3,   300] loss: 0.257
[4,    20] loss: 0.331
[4,    40] loss: 0.283
[4,    60] loss: 0.232
[4,    80] loss: 0.274
[4,   100] loss: 0.283
[4,   120] loss: 0.266
[4,   140] loss: 0.260
[4,   160] loss: 0.269
[4,   180] loss: 0.243
[4,   200] loss: 0.269
[4,   220] loss: 0.263
[4,   240] loss: 0.279
[4,   260] loss: 0.267
[4,   280] loss: 0.224
[4,   300] loss: 0.252
[5,    20] loss: 0.252
[5,    40] loss: 0.259
[5,    60] loss: 0.255
[5,    80] loss: 0.302
[5,   100] loss: 0.253
[5,   120] loss: 0.240
[5,   140] loss: 0.246
[5,   160] loss: 0.273
[5,   180] loss: 0.234
[5,   200] loss: 0.232
[5,   220] loss: 0.239
[5,   240] loss: 0.216
[5,   260] loss: 0.232
[5,   280] loss: 0.269
[5,   300] loss: 0.252
Accuracy: 90.9%
Precision: 0.9115035878283477
Recall: 0.909
F1 Score: 0.9088613781561755
Parameters:  {'epochs': 5, 'layers': [16, 32, 64, 128, 256]}
[1,    20] loss: 0.735
[1,    40] loss: 0.556
[1,    60] loss: 0.464

[1,    80] loss: 0.501
[1,   100] loss: 0.473
[1,   120] loss: 0.524
[1,   140] loss: 0.436
[1,   160] loss: 0.404
[1,   180] loss: 0.405
[1,   200] loss: 0.389
[1,   220] loss: 0.384
[1,   240] loss: 0.351
[1,   260] loss: 0.342
[1,   280] loss: 0.379
[1,   300] loss: 0.343
[2,    20] loss: 0.345
[2,    40] loss: 0.285
[2,    60] loss: 0.289
[2,    80] loss: 0.303
[2,   100] loss: 0.329
[2,   120] loss: 0.316
[2,   140] loss: 0.323
[2,   160] loss: 0.329
[2,   180] loss: 0.301
[2,   200] loss: 0.291
[2,   220] loss: 0.304
[2,   240] loss: 0.270
[2,   260] loss: 0.310
[2,   280] loss: 0.306
[2,   300] loss: 0.256
[3,    20] loss: 0.289
[3,    40] loss: 0.295
[3,    60] loss: 0.275
[3,    80] loss: 0.279
[3,   100] loss: 0.292
[3,   120] loss: 0.263
[3,   140] loss: 0.307
[3,   160] loss: 0.261
[3,   180] loss: 0.254
[3,   200] loss: 0.288
[3,   220] loss: 0.265
[3,   240] loss: 0.254
[3,   260] loss: 0.269
[3,   280] loss: 0.313
[3,   300] loss: 0.276
[4,    20] loss: 0.279
[4,    40] 

In [40]:
# Assuming best_params contains 'epochs' and 'layers' from your tuning
best_epochs = best_params['epochs']
best_layer_config = best_params['layers']

# Reinitialize the model with the best layer configuration
best_model = CNN(num_classes=2, layers=best_layer_config)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=0.001) 

# Retrain the model using the best parameters
for epoch in range(best_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 20 == 19:  # print every 20 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0

print('Finished Retraining with Best Parameters')

[1,    20] loss: 0.610
[1,    40] loss: 0.429
[1,    60] loss: 0.359
[1,    80] loss: 0.358
[1,   100] loss: 0.327
[1,   120] loss: 0.325
[1,   140] loss: 0.305
[1,   160] loss: 0.339
[1,   180] loss: 0.291
[1,   200] loss: 0.323
[1,   220] loss: 0.298
[1,   240] loss: 0.279
[1,   260] loss: 0.273
[1,   280] loss: 0.313
[1,   300] loss: 0.249
[2,    20] loss: 0.269
[2,    40] loss: 0.258
[2,    60] loss: 0.315
[2,    80] loss: 0.285
[2,   100] loss: 0.287
[2,   120] loss: 0.310
[2,   140] loss: 0.292
[2,   160] loss: 0.232
[2,   180] loss: 0.277
[2,   200] loss: 0.298
[2,   220] loss: 0.286
[2,   240] loss: 0.267
[2,   260] loss: 0.279
[2,   280] loss: 0.311
[2,   300] loss: 0.263
[3,    20] loss: 0.248
[3,    40] loss: 0.310
[3,    60] loss: 0.278
[3,    80] loss: 0.291
[3,   100] loss: 0.291
[3,   120] loss: 0.240
[3,   140] loss: 0.244
[3,   160] loss: 0.280
[3,   180] loss: 0.260
[3,   200] loss: 0.246
[3,   220] loss: 0.255
[3,   240] loss: 0.277
[3,   260] loss: 0.244
[3,   280] 

In [41]:
# Path to where model parameters are saved
PATH = './cnn.pth'

In [42]:
# Save parameters
torch.save(best_model.state_dict(), PATH)

In [43]:
# Load model parameters (can skip training)
model = CNN(layers=[16, 32])

model.load_state_dict(torch.load(PATH))

model.eval()

CNN(
  (layers): ModuleList(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=20000, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [44]:
# Testing of validation data
true_labels = []
predictions = []

with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Accuracy
correct = (predictions == true_labels).sum().item()
total = true_labels.size
accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy}%')

# Precision, recall, and f1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')


Accuracy of the model on the test images: 91.7%
Precision: 0.9172019257320543
Recall: 0.917
F1 Score: 0.9169899557846499
